In [1]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import pickle
import re
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.cluster import KMeans, k_means_, DBSCAN, AgglomerativeClustering
from sklearn import cluster
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_distances

import hdbscan
from gensim import models
from gensim.corpora import mmcorpus, Dictionary
from gensim.models import lsimodel, ldamodel, tfidfmodel, rpmodel, logentropy_model, TfidfModel, LsiModel, LdaModel
from gensim import matutils, corpora
from gensim.models.doc2vec import TaggedDocument
from gensim.models import doc2vec
from gensim.models.word2vec import Word2Vec
from sklearn.svm import SVC

import sys
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import html
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import chat_bot as cb
import Database_Handler as dh
from datetime import datetime
import seaborn as sns
import functools
from konlpy.tag import Mecab
from konlpy.utils import pprint
mecab = Mecab()
%matplotlib inline

In [2]:
dataDict = pickle.load(open('./data/pre_data/for_statistics_daum_from_mongodb.pickled','rb'))
print (len(dataDict))

4722


In [3]:
keywordsDict = pickle.load(open('./data/pre_data/keywords_daum.pickled','rb'))

In [4]:
for idx in dataDict:
    dataDict[idx]['extracted_keywords'] = keywordsDict[idx]

In [5]:
df_daum = pd.DataFrame.from_dict(dataDict, orient='index')
df_daum['date'] = pd.to_datetime(df_daum['date']).dt.date
df_daum.reset_index(inplace = True)
df_daum.rename(columns={'index':'id'}, inplace=True)
print (df_daum.shape)
df_daum.head()

(4722, 11)


,id,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords
0,5a2a61bf588c13481c229d1e,뉴스,2017-12-07,세계일보,1093,911,1,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, ...","[부작용, 비아그라, 아일랜드]","{건강, 공장, 세보 효과, 남성들, 연기, 지역, 부작용}"
1,5a2a61bf588c13481c229d1f,뉴스,2017-12-07,헬스조선,603,386,2,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,점심식사를 마친 후 후식으로 커피를 마시는 사람들이 많다. 실제로 직장이 밀집돼 ...,"[커피, 낮잠, 음식물]","{건강, 치아, 낮잠, 커피, 철분, 디스크, 자세, 점심 식사, 식후, 입냄새}"
2,5a2a61bf588c13481c229d20,뉴스,2017-12-07,연합뉴스,1067,811,3,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""",(성남=연합뉴스) 최해민 기자 = 십년지기 지인을 산 채로 묻어 살해한 50대 여...,"[살인혐의, 철원, 검찰송치]","{앙심, 성관계, 주변, 남편, 철원, 지인, 진술, 범행, 경찰, 아들}"
3,5a2a61bf588c13481c229d21,뉴스,2017-12-07,헤럴드경제,418,369,4,"신영자, 억 소리나는 갑질","신영자, 적용안된 혐의→검찰 상고에서 인정\n신영자, 얼마를 어떻게 받았나 [헤럴...","[신영자, 갑질, 롯데백화점]","{롯데, 매장, 검찰, 신영자 이사장, 유통업체, 혐의, 네이처리퍼블릭, 징역}"
4,5a2a61bf588c13481c229d22,뉴스,2017-12-07,연합뉴스,434,368,5,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지",(청주=연합뉴스) 이승민 기자 = 지난 9월 청주의 한 하천에서 20대 여성을 둔기...,"[공범, 살인, 과자]","{쪽지, 과자, 폭행, 혐의, 남자친구, 유치장, 범행, 경찰}"


In [6]:
textInfo = df_daum.title.values + df_daum.mainText.values

In [7]:
def nav_tokenizer(corpus):
    pos = mecab.pos(corpus)
    res = [x[0] for x in pos if (x[1] == u'VV' or x[1] == u'VA' or x[1] == u'NNB' or x[1] == u'NNP' or x[1] == u'NNG')]
    return res

In [8]:
def remove_stopwords(words, stopwords):
    res = [x for x in words if x not in stopwords]
    res = [x for x in words if len(x) != 1]
    return res

In [9]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

### doc2vec

In [10]:
d2v_docs = [nav_tokenizer(x) for x in textInfo]
d2v_docs = [remove_stopwords(x, (stopwords)) for x in d2v_docs]

In [11]:
tagged_d2v_docs = [TaggedDocument(doc, [idx]) for idx, doc in enumerate(d2v_docs)]

In [12]:
d2v_model = doc2vec.Doc2Vec(size=500,dm_mean=1, workers = 2, alpha=0.025, min_alpha=0.025, seed=0, iter = 10, window=8)
d2v_model.build_vocab(tagged_d2v_docs)
d2v_model.train_lbls = False # do not train labels of words
d2v_model.train_words = True # only train relations among words

In [160]:
for epoch in range(30):
    d2v_model.train(tagged_d2v_docs, total_examples=len(d2v_docs), epochs=epoch,)
    d2v_model.alpha -= 0.002 # decrease the learning rate
    d2v_model.min_alpha = d2v_model.alpha # fix the learning rate, no decay

In [161]:
d2v_model.save('./data/pre_data/pre_data_classification_size1500_epoch30_iter10_window5_doc2vec_model_by_mecab.d2v')

In [162]:
X_d2v = [d2v_model.infer_vector(x.words) for x in tagged_d2v_docs]

In [163]:
print(d2v_model.most_similar('신혜선'))

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('황금빛', 0.6773727536201477), ('박시후', 0.6616433262825012), ('조우리', 0.6040634512901306), ('유인영', 0.5991173386573792), ('천호진', 0.5955693125724792), ('신현수', 0.5942219495773315), ('서지안', 0.5876052379608154), ('김예령', 0.5846760272979736), ('이다인', 0.5805894136428833), ('서은수', 0.5775101184844971)]


In [164]:
print(d2v_model.most_similar('손흥민'))

[('토트넘', 0.6700240969657898), ('선수', 0.6664606928825378), ('곽현수', 0.6543377637863159), ('김수진', 0.6475198268890381), ('경기', 0.6340723037719727), ('계약', 0.6281232833862305), ('마우리시오', 0.6276405453681946), ('시즌', 0.6197263598442078), ('KIA타이거즈', 0.619143545627594), ('한국', 0.6154860854148865)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [165]:
print(d2v_model.most_similar('황금빛내인생'))

[('마이웨이', 0.7040160298347473), ('장승조', 0.69057297706604), ('첫눈', 0.6836873888969421), ('유동근', 0.6762100458145142), ('투깝스', 0.6707916259765625), ('야화', 0.6680173873901367), ('진형욱', 0.6532869338989258), ('막영애', 0.6478058695793152), ('원미경', 0.647783100605011), ('김희원', 0.6429933905601501)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [166]:
print(d2v_model.most_similar('홍준표'))

[('척당불기', 0.6043415665626526), ('친일', 0.591914713382721), ('맹비난', 0.580083966255188), ('김동철', 0.5785180926322937), ('대표', 0.577202558517456), ('류여해', 0.5737533569335938), ('하지마', 0.5685048699378967), ('대구', 0.5655708909034729), ('한국당', 0.5631750226020813), ('곽현수', 0.5613089203834534)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [167]:
print(d2v_model.most_similar('문재인'))

[('곽현수', 0.6666592359542847), ('서울시장', 0.657971978187561), ('정원일', 0.644519031047821), ('한국미디어네트워크', 0.6399348378181458), ('친일', 0.6319478154182434), ('방송', 0.6316596269607544), ('김수진', 0.6312470436096191), ('정제', 0.6306769847869873), ('박근혜', 0.6298050284385681), ('안승호', 0.6289332509040833)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [168]:
print(d2v_model.most_similar('노무현'))

[('김영삼', 0.6015615463256836), ('김대중', 0.5935829877853394), ('기록물', 0.5773419141769409), ('정봉주', 0.5681999921798706), ('피살', 0.5581229329109192), ('조화', 0.5565639734268188), ('업적', 0.5531013607978821), ('억압', 0.5495105385780334), ('이재용', 0.5493960976600647), ('참모', 0.5491175055503845)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [169]:
print(d2v_model.most_similar('박근혜'))

[('문재인', 0.6298050880432129), ('곽현수', 0.6087371110916138), ('서울시장', 0.5995140671730042), ('친일', 0.5938601493835449), ('김성현', 0.5705930590629578), ('이명박', 0.570553719997406), ('재조명', 0.567793607711792), ('정제', 0.5659745931625366), ('정원일', 0.564014732837677), ('경기', 0.563276469707489)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [170]:
print(d2v_model.most_similar('국정원'))

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('임관빈', 0.6096518635749817), ('불법사찰', 0.6019033193588257), ('정무수석', 0.5994888544082642), ('이병기', 0.5959900617599487), ('이첩', 0.5858913660049438), ('총무', 0.5804983377456665), ('국정원장', 0.5791335701942444), ('지시', 0.5785329937934875), ('특활비', 0.5764080286026001), ('문건', 0.5761609673500061)]


In [171]:
print(d2v_model.most_similar('이낙연'))

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('국무총리', 0.7945094704627991), ('우원식', 0.6650528907775879), ('합동', 0.6253543496131897), ('김동철', 0.6172805428504944), ('정부서울청사', 0.617144763469696), ('신년인사회', 0.6077857613563538), ('이병기', 0.6031727194786072), ('장례식장', 0.5963466167449951), ('정봉주', 0.5944423675537109), ('타워', 0.5920003056526184)]


In [172]:
print(d2v_model.most_similar('이국종'))

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('아주대병원', 0.6238072514533997), ('교수', 0.5766035914421082), ('귀순병', 0.5763857960700989), ('병사', 0.5708316564559937), ('중증외상센터', 0.5646792054176331), ('소령', 0.5493388175964355), ('센터장', 0.5483399629592896), ('건국대', 0.5328307747840881), ('외상', 0.5315074920654297), ('박형준', 0.531294584274292)]


In [173]:
print(d2v_model.most_similar('김성태'))

[('스포츠한국', 0.8747864365577698), ('이용균', 0.8663179278373718), ('박상경', 0.8650885224342346), ('MBC뉴스', 0.8622350096702576), ('이재호', 0.8616690635681152), ('문수연', 0.8608673810958862), ('박소영', 0.8600618243217468), ('오효진', 0.8588403463363647), ('이가영', 0.8588307499885559), ('박재호', 0.8561685085296631)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [174]:
print(d2v_model.most_similar('더불어민주당'))

[('정의당', 0.6652736663818359), ('이정미', 0.6306555867195129), ('정론관', 0.6284468770027161), ('대한애국당', 0.6260098218917847), ('친일', 0.6179811358451843), ('민주당', 0.6108694672584534), ('김동철', 0.6106832027435303), ('국민의당', 0.6090357899665833), ('정제', 0.6023514866828918), ('서울시장', 0.5895410180091858)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [175]:
print(d2v_model.most_similar('자유한국당'))

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('서울시장', 0.7494426965713501), ('안승호', 0.7407677173614502), ('문수지', 0.7396319508552551), ('곽현수', 0.7369765639305115), ('김성태', 0.7241705060005188), ('오버액션', 0.7209824323654175), ('허정민', 0.7195025086402893), ('정제', 0.7160211205482483), ('한국미디어네트워크', 0.7159568071365356), ('편집부', 0.7132601737976074)]


In [176]:
print(d2v_model.most_similar('국민의당'))

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('곽현수', 0.675626277923584), ('정당', 0.6573776006698608), ('친일', 0.6487647294998169), ('KIA타이거즈', 0.6344984173774719), ('안승호', 0.6330884695053101), ('국회', 0.6330348253250122), ('서울시장', 0.6326565742492676), ('종지부', 0.6309316158294678), ('안철수', 0.6240108609199524), ('대표', 0.6232913136482239)]


In [177]:
print(d2v_model.most_similar('새누리당'))

[('총선', 0.6399968266487122), ('포위', 0.6383978724479675), ('한나라당', 0.628928005695343), ('이우호', 0.6218919157981873), ('서청원', 0.6007388234138489), ('광역', 0.5984536409378052), ('천정배', 0.5948818325996399), ('정경미', 0.5947073698043823), ('신년인사회', 0.5896410942077637), ('경남지사', 0.5839948654174805)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [178]:
print(d2v_model.most_similar('안철수'))

[('국민의당', 0.6240108609199524), ('친일', 0.556397020816803), ('계란', 0.5481189489364624), ('서울시장', 0.5339131355285645), ('오버액션', 0.5306268930435181), ('통합', 0.525336503982544), ('유승민', 0.5190092921257019), ('월간', 0.5177761316299438), ('정당', 0.5122881531715393), ('바로가기', 0.5122097134590149)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [179]:
print(d2v_model.most_similar('유승민'))

[('바른정당', 0.6890402436256409), ('지지층', 0.5561556220054626), ('조원진', 0.5481034517288208), ('야권', 0.5444138050079346), ('보수층', 0.5418297052383423), ('오신환', 0.5317832231521606), ('안대희', 0.52724689245224), ('지지율', 0.5265429615974426), ('미드필드진', 0.5240938663482666), ('한국프로축구연맹', 0.5190296769142151)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [180]:
print(d2v_model.most_similar('박지성'))

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('축구대표팀', 0.585304856300354), ('선수', 0.5773727297782898), ('사이타마', 0.5732486248016357), ('맨유', 0.5725869536399841), ('유나이티드', 0.5722129940986633), ('잉글랜드', 0.5673849582672119), ('축구', 0.5543226003646851), ('시즌', 0.5521440505981445), ('맨체스터', 0.5491272211074829), ('마우리시오', 0.5449378490447998)]


### word2vec

In [ ]:
w2v_model = Word2Vec(d2v_docs, sg=1, size=3000,window=8, workers = 2, iter = 10)
w2v_model.init_sims(replace = True)

In [ ]:
print(w2v_model.most_similar('신혜선'))

In [ ]:
print(w2v_model.most_similar('손흥민'))

In [ ]:
print(w2v_model.most_similar('황금빛내인생'))

In [ ]:
print(w2v_model.most_similar('홍준표'))

In [ ]:
print(w2v_model.most_similar('문재인'))

In [ ]:
print(w2v_model.most_similar('노무현'))

In [ ]:
print(w2v_model.most_similar('박근혜'))

In [ ]:
print(w2v_model.most_similar('국정원'))

In [ ]:
print(w2v_model.most_similar('이낙연'))

In [ ]:
print(w2v_model.most_similar('이국종'))

In [ ]:
print(w2v_model.most_similar('김성태'))

In [ ]:
print(w2v_model.most_similar('더불어민주당'))

In [ ]:
print(w2v_model.most_similar('새누리당'))

In [ ]:
print(w2v_model.most_similar('자유한국당'))

In [ ]:
print(w2v_model.most_similar('국민의당'))

In [ ]:
print(w2v_model.most_similar('안철수'))

In [ ]:
print(w2v_model.most_similar('유승민'))

In [ ]:
print(w2v_model.most_similar('박지성'))

In [ ]:
w2v_model.save('./data/pre_data/word2vec_category_size3000_window8_iter10_by_mecab.w2v')

In [ ]:
embedding_matrix = np.zeros((len(w2v_model.wv.vocab), 3000))
for i in range(len(w2v_model.wv.vocab)):
    embedding_vector = w2v_model.wv[model.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, embeddings, merge

In [ ]:
valid_size = 16  # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
# input words - in this case we do sample by sample evaluations of the similarity
valid_word = Input((1,), dtype='int32')
other_word = Input((1,), dtype='int32')
# setup the embedding layer
embeddings = Embedding(input_dim=embedding_matrix.shape[0], output_dim=embedding_matrix.shape[1],
                      weights=[embedding_matrix])
embedded_a = embeddings(valid_word)
embedded_b = embeddings(other_word)
similarity = merge([embedded_a, embedded_b], mode='cos', dot_axes=2)
# create the Keras model
k_model = Model(input=[valid_word, other_word], output=similarity)

def get_sim(valid_word_idx, vocab_size):
    sim = np.zeros((vocab_size,))
    in_arr1 = np.zeros((1,))
    in_arr2 = np.zeros((1,))
    in_arr1[0,] = valid_word_idx
    for i in range(vocab_size):
        in_arr2[0,] = i
        out = k_model.predict_on_batch([in_arr1, in_arr2])
        sim[i] = out
    return sim

# now run the model and get the closest words to the valid examples
for i in range(valid_size):
    valid_word = w2v_model.wv.index2word[valid_examples[i]]
    top_k = 8  # number of nearest neighbors
    sim = get_sim(valid_examples[i], len(w2v_model.wv.vocab))
    nearest = (-sim).argsort()[1:top_k + 1]
    log_str = 'Nearest to %s:' % valid_word
    for k in range(top_k):
        close_word = w2v_model.wv.index2word[nearest[k]]
        log_str = '%s %s,' % (log_str, close_word)
    print(log_str)